In [1]:
import pandas as pd
df = pd.read_csv('./data/combined_transactions_categorized.csv')

In [2]:
# Generate analysis text

analysis = f"""
FINANCE ANALYSIS REPORT

Total Transactions Analyzed: {len(df)}
Date Range: {df['Date'].min()} to {df['Date'].max()}

INCOME ANALYSIS:
Total Income: €{df[df['Expense/Income']=='Income']['Amount'].sum():,.2f}

EXPENSE ANALYSIS:
Total Expenses: €{df[df['Expense/Income']=='Expense']['Amount'].sum():,.2f}

TOP SPENDING CATEGORIES:
{df[df['Expense/Income']=='Expense'].groupby('Category').agg({'Amount':'sum'}).sort_values(by='Amount', ascending=False).head(10).to_string()}


MONTHLY BREAKDOWN:
{df.groupby(df['Date'].str.slice(0,7)).agg({'Amount':'sum'}).rename(columns={'Amount':'Net Amount'}).to_string()}


LARGEST TRANSACTIONS:
{df.nlargest(20, 'Amount')[['Date', 'Name / Description', 'Amount', 'Category']].to_string()}
"""


# save analysis to a text file
with open('finance_analysis_report.txt','w') as f:
    f.write(analysis)
    
print("Finance analysis report generated: finance_analysis_report.txt")



Finance analysis report generated: finance_analysis_report.txt
